# Easy use

This file allows you to use the entire software package yourself, with minimal programming!

In [2]:
%matplotlib inline
import sys
import os

# Sort the paths out to run from this file
parent_path = os.path.dirname(os.getcwd())
src_path = os.path.join(os.path.sep,parent_path, 'src')
sys.path.append(src_path)
print(src_path)

/home/dharrington/cyclops/src


## Step 1 - simulation to csv

In [3]:
from read_exodus import *


reader = ExodusReader('monoblock_out.e')
pickle_manager = PickleManager()

sensor_region = 'right'
pos_3D = reader.read_pos(sensor_region)
temps = reader.read_scalar(sensor_region, 'temperature')

pos_2D = compress_2D(pos_3D)
bounds = find_bounds(pos_2D)
grid = generate_grid(bounds, 30, 30)

temp_field = ScalarField(LModel, bounds, 2)
temp_field.fit_model(pos_2D, temps)

pickle_manager.save_file('simulation', 'field_temp.obj', temp_field)
pickle_manager.save_file('simulation', 'grid.obj', grid)


Reading file...
<meshio mesh object>
  Number of points: 87200
  Number of cells:
    hexahedron27: 2016
    hexahedron27: 3360
    hexahedron27: 4704
  Point sets: right, top, left, , centre_x_bottom_y_back_z, centre_x_bottom_y_front_z, left_x_bottom_y_centre_z, right_x_bottom_y_centre_z
  Point data: disp_x, disp_y, disp_z, temperature
  Cell data: vonmises_stress
